In [9]:
# Importing all the required Libraries

import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential                                                            
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization,Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np 
import cv2
import os

In [10]:
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

In [11]:
train_path = '../input/face-expression-recognition-dataset/images/train'

In [12]:
val_path = '../input/face-expression-recognition-dataset/images/validation'

In [13]:
os.listdir(train_path)

In [14]:
os.listdir(val_path)

In [15]:
categories = os.listdir(train_path)

In [ ]:
# Creating a function for using for show some images from each categories
def imageshow(category):
  plt.figure(figsize= (8,8))
  for i in range(1, 10, 1):
      plt.subplot(3,3,i)
      img = load_img(train_path+'/'+category+"/"+
                    os.listdir(train_path + "/" + category)[i], target_size=(48,48))
      plt.imshow(img)
  plt.suptitle(category,fontsize=30)   
  plt.show()

In [ ]:
#Showing some images from category neutral
imageshow('neutral')

In [ ]:
#Showing some images from category angry
imageshow('angry')

In [ ]:
#Showing some images from category sad
imageshow('sad')

In [ ]:
#Showing some images from category surprise
imageshow('surprise')

In [ ]:
#Showing some images from category happy
imageshow('happy')

In [ ]:
#Showing some images from category disgust
imageshow('disgust')

In [ ]:
#Showing some images from category fear
imageshow('fear')

# Showing some images from validation set

In [ ]:
# validation set images
for category in categories:
    plt.figure(figsize= (8,8))
    for j in range(1,10,1):
        
        plt.subplot(3,3,j)
        
        img = load_img(val_path+'/'+category+"/"+
                    os.listdir(val_path + "/" + category)[j], target_size=(48,48))
        plt.imshow(img)
    plt.suptitle(category,fontsize=30)
    plt.show()

# **Count of images in each category**

In [ ]:
# count of training images
for expression in os.listdir(train_path):
    print(str(len(os.listdir(train_path +'/' +expression))) + " " + expression + " images")

In [ ]:
# count of validation images
for expression in os.listdir(val_path):
    print(str(len(os.listdir(val_path +'/' +expression))) + " " + expression + " images")

# Data prepration for Self constructed models

In [16]:
# Data Augumentation
train_scale = ImageDataGenerator(rescale=1./255)
    
val_scale = ImageDataGenerator(rescale=1./255)

In [17]:
batch_size  = 64

train_data = train_scale.flow_from_directory(train_path,
                                  target_size = (48,48),
                                  color_mode = "grayscale",
                                  batch_size=batch_size,
                                  class_mode='categorical',
                                  shuffle=True)

val_data = val_scale.flow_from_directory(val_path,
                                  target_size = (48,48),
                                  color_mode = "grayscale",
                                  batch_size=batch_size,
                                  class_mode='categorical',
                                  shuffle=True)

> # **Model Building**

#  **Self constructed model**

In [ ]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))



emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
emotion_model.summary()

In [ ]:
epochs=50
steps_per_epoch=train_data.n//train_data.batch_size
print('train',steps_per_epoch)
validation_steps=val_data.n//val_data.batch_size
print('validation',validation_steps)

In [ ]:
checkpoint = ModelCheckpoint('Emotion_self_cons.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

In [ ]:
# Fitting the model 
history = emotion_model.fit_generator(
    train_data,
    epochs=epochs,
    validation_data = val_data,
    callbacks=callbacks,
    verbose=1)

In [ ]:
emotion_model.save('self_cons_model.h5')    

In [ ]:
import pandas as pd
# saving the history of the model in data frame 
df=pd.DataFrame(emotion_model.history.history)

In [ ]:
df.head()

In [ ]:
#Plotting Accuracy & Loss
plt.style.use('dark_background')

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(df['loss'], label='Training Loss')
plt.plot(df['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(df['accuracy'], label='Training Accuracy')
plt.plot(df['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# compute predictions
predictions = emotion_model.predict_generator(generator=val_data)
y_pred = [np.argmax(probas) for probas in predictions]
y_test = val_data.classes
class_names = val_data.class_indices.keys()

In [ ]:
# show the confusion matrix of our predictions
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')# **Live Class Monitoring System(Face Emotion Recognition)**
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()

# **Data prepration for Transfer learning**

In [18]:
img_size = 224
batch_size = 32

# selecting colour mode as rgb as transfer learning is trained on rgb photos and we have grascaled images
datagen_train = ImageDataGenerator(horizontal_flip=True,brightness_range=[0.8,1.2],rescale=1./255)
train_generator = datagen_train.flow_from_directory(train_path,
                                                  target_size=(img_size,img_size),
                                                  batch_size=batch_size,
                                                  shuffle=True,
                                                  color_mode='rgb',
                                                  class_mode='categorical')

datagen_validation = ImageDataGenerator(horizontal_flip=True,brightness_range=[0.8,1.2],rescale=1./255)
validation_generator = datagen_train.flow_from_directory(val_path,
                                                  target_size=(img_size,img_size),
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  color_mode='rgb',
                                                  class_mode='categorical')

# **Model Vgg16**

In [19]:
#using pretrained model, VGG16 architecture
from tensorflow.keras.applications.vgg16 import VGG16

In [30]:
# creating a base model using resnet and loading the pretrained weights
base_model = VGG16(input_shape=(224,224,3),include_top = False, weights = 'imagenet')
base_model.summary()

In [31]:
# making all the layers except last layer non trainable 
for layer in base_model.layers:
    layer.trainable = False

In [32]:
# our layers - you can add more if you want
x = Flatten()(base_model.output)

In [33]:
prediction = Dense(7, activation='softmax')(x)

In [34]:
# create a model object
model = Model(inputs=base_model.input, outputs=prediction)

In [35]:
model.summary()

In [36]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [37]:
epochs=50
steps_per_epoch=train_generator.n//train_generator.batch_size
print(steps_per_epoch)
validation_steps=validation_generator.n//validation_generator.batch_size
print(validation_steps)

In [38]:
"""
I used two callbacks one is `early stopping` for avoiding overfitting training data
and other `ReduceLROnPlateau` for learning rate.
"""
# this decreases the learning rate if the model loss does not decrease 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=10, min_lr=0.00001, mode='auto')
# to save model weights 
checkpoint = ModelCheckpoint("model_weights_vgg16.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
early_stopping=EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=10,
                            verbose=1,
                            restore_best_weights=True)
callbacks = [checkpoint, reduce_lr,early_stopping]

In [40]:
history = model.fit_generator(
  train_generator,
  validation_data=validation_generator,
  epochs=50,
  steps_per_epoch=steps_per_epoch,
  validation_steps=validation_steps,
callbacks=callbacks
)

# **MObileNet**

In [21]:
from tensorflow.keras.applications.mobilenet import MobileNet

In [22]:
mobile_net = MobileNet(input_shape = (224,224,3),include_top = False,weights = "imagenet")

In [30]:
# making all the layers except last layer non trainable 
for layer in mobile_net.layers:
    layer.trainable = False


In [31]:
# our layers - you can add more if you want
x = Flatten()(mobile_net.output)
prediction = Dense(7, activation='softmax')(x)
# create a model object
model = Model(inputs=mobile_net.input, outputs=prediction)

In [25]:
# from tensorflow.keras.layers import Flatten, Dense, GlobalAvgPool2D, GlobalMaxPool2D

# x = mobile_net.layers[-14].output

# global_pool = GlobalMaxPool2D(name="global_pool")(x)

# output = Dense(7, activation="softmax", name="out_layer")(global_pool)

# model = Model(inputs=mobile_net.input, outputs=output)

In [32]:
model.summary()

In [33]:
from tensorflow.keras import optimizers

optims = [
    optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
    optimizers.Adam(0.01),
]

model.compile(
        loss='categorical_crossentropy',
        optimizer=optims[1],
        metrics=['accuracy']
)

In [35]:
"""
I used two callbacks one is `early stopping` for avoiding overfitting training data
and other `ReduceLROnPlateau` for learning rate.
"""
# this decreases the learning rate if the model loss does not decrease 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=10, min_lr=0.00001, mode='auto')
# to save model weights 
checkpoint = ModelCheckpoint("model_weights_mobile_net.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
early_stopping=EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=10,
                            verbose=1,
                            restore_best_weights=True)
callbacks = [checkpoint, reduce_lr,early_stopping]

In [54]:
"""
I used two callbacks one is `early stopping` for avoiding overfitting training data
and other `ReduceLROnPlateau` for learning rate.
"""
# early_stopping = EarlyStopping(
#     monitor='val_accuracy',
#     min_delta=0.00008,
#     patience=11,
#     verbose=1,
#     restore_best_weights=True,
# )
# # to save model weights 
# checkpoint = ModelCheckpoint("model_weights_mobilenet.h5", monitor='val_accuracy',
#                              save_weights_only=True, mode='max', verbose=1)
# lr_scheduler = ReduceLROnPlateau(
#     monitor='val_accuracy',
#     min_delta=0.0001,
#     factor=0.25,
#     patience=4,
#     min_lr=1e-7,
#     verbose=1,
# )

# callbacks = [
#     early_stopping,
#     lr_scheduler,
#     checkpoint
# ]

In [37]:
epochs=50
steps_per_epoch=train_generator.n//train_generator.batch_size
print(steps_per_epoch)
validation_steps=validation_generator.n//validation_generator.batch_size
print(validation_steps)

In [38]:
history = model.fit_generator(
  train_generator,
  validation_data=validation_generator,
  epochs=50,
  steps_per_epoch=steps_per_epoch,
  validation_steps=validation_steps,
callbacks=callbacks
)

In [39]:
model.save('model')

In [41]:
model.save('mobile_net.h5')

In [43]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from time import sleep
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
import tensorflow as tf

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') # Face Detection
classifier =load_model("./mobile_net.h5")  #Load model

emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']  # Emotion that will be predicted

cap = cv2.VideoCapture(0)  ## Opening webcam



while True:
    _, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)  ##Face Cropping for prediction



        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0) ## reshaping the cropped face image for prediction

            prediction = classifier.predict(roi)[0]   #Prediction
            label=emotion_labels[prediction.argmax()]
            label_position = (x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)   # Text Adding
        else:
            cv2.putText(frame,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()